In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import numpy
CC = ComplexField(32)
pi = CC.pi()
I = CC.gen()

def generate_random_hnp_instance(leak_size, number_of_signatures, prime, key):
    #h = k - cx
    HNP_instance = []
    bound = (prime-1) / 2**(leak_size+1)

    for j in range(number_of_signatures):
        c = randrange(1, prime)
        k = randrange(-round(bound), floor(bound))
        h = (k - c*key) % prime
        HNP_instance.append([c, h])
    return HNP_instance

def generate_hnp_instance(leak_size, number_of_signatures, prime, key):
    #h = k - cx
    hnp_instance = []

    bit_length = ceil(log(prime, 2))
    bound = 2**(bit_length-leak_size-1) #-1 protoze centerujeme k

    for j in range(number_of_signatures):
        c = randrange(1, prime)
        k = randrange(-round(bound), floor(bound))
        h = (c*key - k) % prime
        hnp_instance.append([c, h])
    return hnp_instance


def sort_and_difference(hnp_instance, prime, number_of_iterations, bound_exponent):
    new_instance = hnp_instance
    for i in range(number_of_iterations):
        new_instance.sort(key=lambda x:x[0])
        helper_list = []
        for j in range(len(new_instance)-1):
            if i < len(new_instance)-2 | (new_instance[j+1][0]-new_instance[j][0]) < 2**bound_exponent:
                helper_list.append(((new_instance[j+1][0]-new_instance[j][0]), (new_instance[j+1][1]-new_instance[j][1])% prime))
        new_instance = helper_list.copy()
    return new_instance

def bleichenbacher(hnp_instance, prime, bound_exponent):
    z = [0] * (2**bound_exponent + 1)
    for i in range(len(hnp_instance)-1):
        z[hnp_instance[i][0]] += (1/len(hnp_instance)) * exp(2*pi*I* hnp_instance[i][1]/prime)
    frequencies = numpy.fft.ifft(z)
    magnitudes = numpy.abs(frequencies)
    
    return numpy.argmax(magnitudes)*prime/(2**(bound_exponent))



#def build_Kannan_embedding(hnp_instance, prime, leak):
    

def transform_hnp_instance_to_lattice_reduction_form(hnp_instance, prime):
    for i in range(len(hnp_instance)):
        hnp_instance[i][0] = (-1 * hnp_instance[i][0]) % prime

def nearest_plane_recursive(basis, GM_basis, x, dimension):
    dimension -= 1

    if dimension == 0:
        delta = round(x *  GM_basis.column(dimension))
        return delta * basis.column(dimension)

    gamma_d = x *  GM_basis.column(dimension)
    
    delta = round(gamma_d)
    
    v = delta * basis.column(dimension)
    x_prime = x - v
    
    y = nearest_plane_recursive(basis, GM_basis, x_prime, dimension)
    
    return y + v




In [178]:
import math

LEAK_SIZE = 1
NUMBER_OF_SIGNATURES = 5000
NUMBER_OF_RECOVERED_MSB = 15 #maximalni velikost fft

prime = 600979  
key = 500000

print(math.log2(key))

hnp_instance = generate_random_hnp_instance(LEAK_SIZE, NUMBER_OF_SIGNATURES, prime, key)

hnp_instance = sort_and_difference(hnp_instance, prime, 2, NUMBER_OF_RECOVERED_MSB)

msb_of_key = bleichenbacher(hnp_instance, prime, NUMBER_OF_RECOVERED_MSB)

print(msb_of_key)
print("the original key is: ", bin(key))
print("we recovered:", bin(int(round(key - msb_of_key))))

18.931568569324174
455392.71759033203
the original key is:  0b1111010000100100000
we recovered: 0b1010111000111111


In [ ]:
LEAK_SIZE = 3
NUMBER_OF_SIGNATURES = 100

prime = 600979 
key = 500000

hnp_instance = generate_hnp_instance(LEAK_SIZE, NUMBER_OF_SIGNATURES, prime, key)

ring = Integers(prime)
t_0 = ring(hnp_instance[0][0])
t_0 = t_0.inverse()


hnp_instance_eliminated_key = []
target = [0] * (len(hnp_instance) + 1)
for i in range(len(hnp_instance)-1):
    c = hnp_instance[i+1][0] * t_0
    h = hnp_instance[i+1][1] - hnp_instance[i+1][0] * t_0 * hnp_instance[0][1]
    hnp_instance_eliminated_key.append([c, h])


#build kannan embedding
basis = matrix(len(hnp_instance_eliminated_key) + 2)
basis[len(hnp_instance_eliminated_key), len(hnp_instance_eliminated_key)] = 1
basis[len(hnp_instance_eliminated_key) + 1, len(hnp_instance_eliminated_key) + 1] = 1
for i in range(len(hnp_instance_eliminated_key)):
    basis[i,i] = prime
    basis[len(hnp_instance_eliminated_key), i] = hnp_instance_eliminated_key[i][0] 
    basis[len(hnp_instance_eliminated_key) + 1, i] = hnp_instance_eliminated_key[i][1]

reduced_basis = basis.LLL()

k_0 = reduced_basis.row(0)[len(hnp_instance_eliminated_key)]

print("Pak key je:", t_0*(hnp_instance[0][1]-k_0))



Pak key je: 500000
